<a href="https://colab.research.google.com/github/arunravi8595/CE888/blob/main/Lab_3/bank_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder


In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab3/bank-additional-full.csv", delimiter = ";")

features = [
            "age",
            "job",
            "marital",
            "education",
            "default",
            "housing",
            "loan",
            "contact",
            "month",
            "day_of_week",
            "campaign",
            "pdays",
            "previous",
            "poutcome",
            "emp.var.rate",
            "cons.price.idx",
            "cons.conf.idx",
            "euribor3m",
            "nr.employed",
            "y"]

df[features].head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [15]:
outcomes=  ["age",
            "job",
            "marital",
            "education",
            "default",
            "housing",
            "loan",
            "contact",
            "month",
            "day_of_week",
            "campaign",
            "pdays",
            "previous",
            "poutcome",
            "emp.var.rate",
            "cons.price.idx",
            "cons.conf.idx",
            "euribor3m",
            "nr.employed",
            "y"]

df[outcomes].head()

print(df[outcomes[:]].head().to_latex())

\begin{tabular}{lrlllllllllrrrlrrrrrl}
\toprule
{} &  age &        job &  marital &    education &  default & housing & loan &    contact & month & day\_of\_week &  campaign &  pdays &  previous &     poutcome &  emp.var.rate &  cons.price.idx &  cons.conf.idx &  euribor3m &  nr.employed &   y \\
\midrule
0 &   56 &  housemaid &  married &     basic.4y &       no &      no &   no &  telephone &   may &         mon &         1 &    999 &         0 &  nonexistent &           1.1 &          93.994 &          -36.4 &      4.857 &       5191.0 &  no \\
1 &   57 &   services &  married &  high.school &  unknown &      no &   no &  telephone &   may &         mon &         1 &    999 &         0 &  nonexistent &           1.1 &          93.994 &          -36.4 &      4.857 &       5191.0 &  no \\
2 &   37 &   services &  married &  high.school &       no &     yes &   no &  telephone &   may &         mon &         1 &    999 &         0 &  nonexistent &           1.1 &          93.994 &     

In [17]:
# convert a string variable to a categorical one
df[["Type"]] = df[["Type"]].apply(LabelEncoder().fit_transform)

df.head()
df_dummies = pd.get_dummies(df)

KeyError: ignored